In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE    

In [3]:
df = pd.read_csv('../timeseries_data_min10.csv',index_col=0)
df.head()

,LH1,LH2,LH3,LH4,LH5,LH6,LH7,LH8,LH9,LH10,...,SC12,SC13,SC14,SC15,SC16,SC17,SC18,SC19,Scene,Run
TR_second,,,,,,,,,,,,,,,,,,,,,
55,0.054701,-0.25006,0.72957,1.14260,0.61746,0.13296,1.1425,-0.025135,0.494680,0.00080,...,0.07187,0.037064,-0.017353,-0.14876,0.030370,0.128870,0.071993,-0.076383,68,1
57,-0.116570,-0.43434,0.63090,1.02230,1.12070,0.11760,1.2013,-0.250950,0.113860,-0.23374,...,-0.06352,-0.092239,-0.137860,-0.28449,-0.071457,0.060694,-0.127740,-0.180910,68,1
59,-0.134580,-0.48107,0.50826,0.80063,1.39880,0.13427,1.3211,-0.332260,-0.140540,-0.36771,...,-0.19368,-0.189810,-0.244960,-0.33239,-0.142480,-0.069656,-0.326870,-0.307720,68,1
61,0.030274,-0.49911,0.49199,0.59911,1.41920,0.20661,1.3933,-0.213830,-0.145100,-0.34612,...,-0.22613,-0.202920,-0.248450,-0.27698,-0.137740,-0.180090,-0.355090,-0.340760,68,1
63,0.156870,-0.64297,0.54619,0.19463,1.16220,0.32851,1.1522,-0.019097,0.033318,-0.16519,...,-0.15575,-0.159270,-0.179240,-0.17226,-0.137520,-0.273580,-0.154300,-0.315850,68,1


In [4]:
X = df.drop(['Scene','Run'],axis=1)
orig_idx = np.array(df.Scene.value_counts().index)
y = [np.where(orig_idx == i)[0][0] for i in df.Scene]

swapper = zip(orig_idx,np.unique(y))

print(X.shape)
print(len(np.unique(y)))

(2956, 419)
59


In [5]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report

def network_modelling(X,y,n_hidden,dropout,test_size=0.2,batch_size=200,epochs=300,seed=474):

    if len(n_hidden)==len(dropout):
        callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=round(epochs/10))

        #one-hot encoded
        y_cat = keras.utils.to_categorical(y, max(np.unique(y))+1) 
        X_train,X_test,y_train,y_test = train_test_split(X,y_cat,test_size=test_size,stratify=y,random_state=seed)

        #SMOTE
        sm = SMOTE(k_neighbors=5)
        X_train_new, y_train_new = sm.fit_sample(X_train,y_train)

        #clear backend
        keras.backend.clear_session()

        #build model
        model = Sequential()
        #input layer
        model.add(Dense(n_hidden[0], activation='relu', bias_initializer='random_normal',
                        input_shape=(X_train_new.shape[1],),name="layer1"))
        model.add(keras.layers.Dropout(dropout[0]))

        #add hidden layers

        for i in range(1,len(n_hidden)):
            model.add(Dense(n_hidden[i], activation='relu',bias_initializer='random_normal',name="layer%d" % (i+1)))
            if dropout[i] > 0:
                model.add(keras.layers.Dropout(dropout[i]))

        #output layer
        model.add(Dense(max(np.unique(y))+1, activation='softmax',name="output"))

        #model.summary()

        #compile
        model.compile(loss='categorical_crossentropy',
                      optimizer=Adam(),
                      metrics=[keras.metrics.Precision(),keras.metrics.Recall()])

        #train model
        history = model.fit(X_train_new, y_train_new,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=0,
                        validation_data=(X_test, y_test),
                        callbacks=[callback])

        #predict
        y_preds = np.argmax(model.predict(X_test),axis=1)
        #y_preds.shape

        #classification report
        report = classification_report(np.argmax(y_test,axis=1), y_preds, output_dict=True)

        #score = model.evaluate(X_test, y_test, verbose=0)
        return report, history, model
    else:
        print('hidden layers and dropout list must have same length')


In [6]:
n_hidden = [210,105]
dropout = [.2,.2]

tt = ''
for i in n_hidden:
    tt += '_%d' % i

results_file = './results_k5%s.csv' % tt

fo = open(results_file,"w+")
fo.write('run,seed,accuracy,precision,recall,early_stopping\n')
fo.close()

rounds = 50

rand_seeds = np.random.randint(0,1000,rounds)

for i,seed in enumerate(rand_seeds):
    print('starting run %s' % i)
    report, history, _ = network_modelling(X,y,n_hidden,dropout,seed=seed)

    #performance metrics
    acc = round(report['accuracy'],4)
    precision = round(report['macro avg']['precision'],4)
    recall = round(report['macro avg']['recall'],4)
    early_stopping = len(history.epoch)

    #print('accuracy: %.2f \nprecision:  %.2f \nrecall: %.2f' % (acc,precision,recall))
    #print('early stopping rounds: %d' % len(history.epoch))
    
    fo = open(results_file,"a")
    fo.write('%d,%d,%s,%s,%s,%d\n' % (i,seed,acc,precision,recall,early_stopping))
    fo.close()
    
    print('run %s completed.' % i)
    print('accuracy: %s, precision:  %s, recall: %s, early stopping rounds: %d\n' % (acc,precision,recall,early_stopping))

starting run 0


C:\Users\Xuan Kai\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


run 0 completed.
accuracy: 0.8936, precision:  0.9089, recall: 0.8374, early stopping rounds: 79

starting run 1
run 1 completed.
accuracy: 0.9358, precision:  0.9539, recall: 0.9299, early stopping rounds: 132

starting run 2
run 2 completed.
accuracy: 0.9172, precision:  0.9316, recall: 0.8944, early stopping rounds: 96

starting run 3
run 3 completed.
accuracy: 0.9341, precision:  0.9483, recall: 0.9131, early stopping rounds: 99

starting run 4
run 4 completed.
accuracy: 0.902, precision:  0.9279, recall: 0.8895, early stopping rounds: 87

starting run 5
run 5 completed.
accuracy: 0.9037, precision:  0.9205, recall: 0.8916, early stopping rounds: 101

starting run 6
run 6 completed.
accuracy: 0.9189, precision:  0.9302, recall: 0.9022, early stopping rounds: 78

starting run 7


C:\Users\Xuan Kai\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


run 7 completed.
accuracy: 0.9223, precision:  0.9295, recall: 0.8877, early stopping rounds: 136

starting run 8
run 8 completed.
accuracy: 0.902, precision:  0.9231, recall: 0.8843, early stopping rounds: 94

starting run 9
run 9 completed.
accuracy: 0.9088, precision:  0.9421, recall: 0.8817, early stopping rounds: 84

starting run 10
run 10 completed.
accuracy: 0.875, precision:  0.9197, recall: 0.8541, early stopping rounds: 82

starting run 11
run 11 completed.
accuracy: 0.9105, precision:  0.9351, recall: 0.8904, early stopping rounds: 94

starting run 12


...

run 28 completed.
accuracy: 0.9054, precision:  0.9154, recall: 0.8764, early stopping rounds: 96

starting run 29
run 29 completed.
accuracy: 0.9223, precision:  0.9354, recall: 0.8882, early stopping rounds: 95

starting run 30
run 30 completed.
accuracy: 0.9426, precision:  0.9622, recall: 0.9338, early stopping rounds: 111

starting run 31
run 31 completed.
accuracy: 0.9206, precision:  0.9484, recall: 0.9362, early stopping rounds: 123

starting run 32
run 32 completed.
accuracy: 0.9054, precision:  0.9241, recall: 0.8883, early stopping rounds: 71

starting run 33
run 33 completed.
accuracy: 0.9088, precision:  0.9175, recall: 0.8773, early stopping rounds: 83

starting run 34
run 34 completed.
accuracy: 0.897, precision:  0.9234, recall: 0.909, early stopping rounds: 87

starting run 35
run 35 completed.
accuracy: 0.9307, precision:  0.9443, recall: 0.9294, early stopping rounds: 103

starting run 36
run 36 completed.
accuracy: 0.9324, precision:  0.928, recall: 0.927, early sto

C:\Users\Xuan Kai\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


run 37 completed.
accuracy: 0.9223, precision:  0.9015, recall: 0.9136, early stopping rounds: 81

starting run 38


C:\Users\Xuan Kai\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


run 38 completed.
accuracy: 0.8885, precision:  0.9033, recall: 0.8778, early stopping rounds: 81

starting run 39
run 39 completed.
accuracy: 0.9122, precision:  0.9226, recall: 0.8903, early stopping rounds: 91

starting run 40
run 40 completed.
accuracy: 0.9088, precision:  0.9186, recall: 0.9033, early stopping rounds: 96

starting run 41
run 41 completed.
accuracy: 0.9139, precision:  0.9241, recall: 0.9135, early stopping rounds: 132

starting run 42
run 42 completed.
accuracy: 0.9206, precision:  0.9326, recall: 0.8826, early stopping rounds: 131

starting run 43
run 43 completed.
accuracy: 0.9139, precision:  0.9314, recall: 0.9245, early stopping rounds: 93

starting run 44
run 44 completed.
accuracy: 0.9375, precision:  0.9427, recall: 0.9233, early stopping rounds: 108

starting run 45
run 45 completed.
accuracy: 0.9071, precision:  0.9251, recall: 0.8971, early stopping rounds: 104

starting run 46
run 46 completed.
accuracy: 0.9358, precision:  0.9729, recall: 0.9374, earl